In [1]:

# Import necessary libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import xgboost as xgb


In [42]:
# file_path = r''
data = pd.read_csv('EDA_Cybersecurity_Attack.csv')
# data = data.drop(columns=['Unnamed: 0'])
# # data_test = data.drop(columns=['Attack Type'])
# # X_test.to_csv('TEST.csv')
# data.to_csv('TEST.csv',index=False)
data["Attack Type"]

0          Malware
1          Malware
2             DDoS
3          Malware
4             DDoS
           ...    
39995         DDoS
39996         DDoS
39997         DDoS
39998      Malware
39999    Intrusion
Name: Attack Type, Length: 40000, dtype: object

In [23]:
selected_features = [
       'Source Port', 'Destination Port', 'Protocol', 'Packet Length',
       'Packet Type', 'Traffic Type', 'Malware Indicators',
       'Anomaly Scores', 'Alerts/Warnings', 'Attack Signature',
       'Action Taken',
       'Network Segment',
       'Firewall Logs', 'IDS/IPS Alerts', 'Log Source',
       'Hour', 'DayOfWeek', 'Month', 'Year', 'Severity level in numeric',
       'Browser', 'Operating System', 'Is Source Private',
       'Is Destination Private']
# Target variable
target = "Attack Type"
# Ensure only selected feature are used
X = data[selected_features]
y= data[target]

# Convert categorical features to 'category' dtype for XGBoost
categorical_cols = ['Protocol', 'Packet Type', 'Traffic Type', 'Action Taken', 'Malware Indicators', 'Alerts/Warnings', 'Attack Signature', 'Network Segment',
                    'Firewall Logs', 'IDS/IPS Alerts', 'Log Source', 'Operating System','Browser']

for col in categorical_cols:
    if col in X.columns:
        X[col] = X[col].astype("category")

print(X.info()) # Check if conversion is sucessfully
# X.to_csv("TEST.csv")


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   Source Port                40000 non-null  int64   
 1   Destination Port           40000 non-null  int64   
 2   Protocol                   40000 non-null  category
 3   Packet Length              40000 non-null  int64   
 4   Packet Type                40000 non-null  category
 5   Traffic Type               40000 non-null  category
 6   Malware Indicators         40000 non-null  category
 7   Anomaly Scores             40000 non-null  float64 
 8   Alerts/Warnings            40000 non-null  category
 9   Attack Signature           40000 non-null  category
 10  Action Taken               40000 non-null  category
 11  Network Segment            40000 non-null  category
 12  Firewall Logs              40000 non-null  category
 13  IDS/IPS Alerts             4000

/var/folders/ch/g7n80fhs1m3b8_1l2vhzh74c0000gn/T/ipykernel_44563/1608925001.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype("category")
/var/folders/ch/g7n80fhs1m3b8_1l2vhzh74c0000gn/T/ipykernel_44563/1608925001.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = X[col].astype("category")
/var/folders/ch/g7n80fhs1m3b8_1l2vhzh74c0000gn/T/ipykernel_44563/1608925001.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [24]:
# Convert target variable to numeric labels
y = y.astype("category").cat.codes

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.2, random_state= 22, stratify= y)


In [27]:
X_test.to_csv

,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Signature,...,Log Source,Hour,DayOfWeek,Month,Year,Severity level in numeric,Browser,Operating System,Is Source Private,Is Destination Private
8225,61727,53971,ICMP,191,Data,DNS,No Data,88.98,No Data,Known Pattern B,...,Firewall,20,2,2,2020,2,Opera,Windows,False,False
14412,23151,17941,TCP,307,Data,HTTP,No Data,10.74,Alert Triggered,Known Pattern A,...,Firewall,10,2,11,2020,3,Mozilla,Windows,False,False
38955,60957,61323,ICMP,606,Data,DNS,IoC Detected,19.05,No Data,Known Pattern A,...,Server,2,3,7,2022,2,Mozilla,Windows,False,False
30346,13905,62976,ICMP,1324,Control,HTTP,IoC Detected,42.75,Alert Triggered,Known Pattern A,...,Server,1,3,6,2021,3,Mozilla,Android,False,False
21193,28936,36922,ICMP,983,Data,DNS,IoC Detected,27.13,No Data,Known Pattern A,...,Firewall,4,4,8,2021,2,Mozilla,MacOS,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26836,46955,24831,UDP,715,Control,FTP,IoC Detected,56.75,No Data,Known Pattern A,...,Firewall,11,3,9,2020,1,Mozilla,MacOS,False,False
39597,28447,19038,ICMP,722,Data,FTP,No Data,76.80,No Data,Known Pattern A,...,Firewall,17,2,11,2022,3,Mozilla,Windows,False,False
28892,5975,23226,TCP,744,Control,HTTP,No Data,85.00,Alert Triggered,Known Pattern B,...,Server,21,1,9,2023,3,Mozilla,MacOS,False,False
11892,55971,21710,ICMP,134,Data,DNS,IoC Detected,1.88,Alert Triggered,Known Pattern A,...,Server,10,6,1,2023,3,Mozilla,MacOS,False,False


In [29]:
X_test.to_csv('TEST.csv',index=False)

In [ ]:
Project1/xgb_model_optimized_model.pkl